In [114]:
import pandas as pd

trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')
trainData.head()

y = trainData['Survived']
trainData.drop(['Survived','Name','Ticket','Cabin'],axis=1,inplace=True)
# trainData.drop('Name',axis=1,inplace=True)

trainData.head(15)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
5,6,3,male,NaN,0,0,8.4583,Q
6,7,1,male,54.0,0,0,51.8625,S
7,8,3,male,2.0,3,1,21.0750,S
8,9,3,female,27.0,0,2,11.1333,S
9,10,2,female,14.0,1,0,30.0708,C


In [115]:
from sklearn.model_selection import train_test_split


missing_val_count_by_column = (trainData.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

# Columns with 'Not Applicable'  in NaN
SetNullToNo = ['Embarked']
#Columns with median in NaN
SetNullToMed = ['Age']


Age         177
Embarked      2
dtype: int64


In [116]:
trainData[SetNullToNo] = trainData[SetNullToNo].fillna(value = 'Not Applicable')
trainData[SetNullToMed] = trainData[SetNullToMed].fillna(trainData[SetNullToMed].median())


testData[SetNullToNo] = testData[SetNullToNo].fillna(value = 'Not Applicable')
testData[SetNullToMed] = testData[SetNullToMed].fillna(testData[SetNullToMed].median())

X_train, X_valid, y_train, y_valid = train_test_split(trainData, y,
                                                    train_size=0.8, test_size=0.2,random_state=0)

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(testData)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)
X_train.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Not Applicable,Embarked_Q,Embarked_S
140,141,3,28.0,0,2,15.2458,True,False,True,False,False,False
439,440,2,31.0,0,0,10.5000,False,True,False,False,False,True
817,818,2,31.0,1,1,37.0042,False,True,True,False,False,False
378,379,3,20.0,0,0,4.0125,False,True,True,False,False,False
491,492,3,21.0,0,0,7.2500,False,True,False,False,False,True
331,332,1,45.5,0,0,28.5000,False,True,False,False,False,True
588,589,3,22.0,0,0,8.0500,False,True,False,False,False,True
358,359,3,28.0,0,0,7.8792,True,False,False,False,True,False
674,675,2,28.0,0,0,0.0000,False,True,False,False,False,True
162,163,3,26.0,0,0,7.7750,False,True,False,False,False,True


In [117]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor


model = XGBRegressor(random_state = 0,n_estimators = 1000,learning_rate = 0.05,n_jobs = 4)

model.fit(X_train,y_train,early_stopping_rounds = 5,eval_set=[(X_valid,y_valid)],verbose = False) # Your code here

# Get predictions
predictions = np.rint(model.predict(X_valid)).astype(int)

# Calculate MAE
mae_3 = mean_absolute_error(predictions,y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)
# predictions


C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Mean Absolute Error: 0.1564245810055866


In [118]:
final_predictions = np.rint(model.predict(X_test)).astype(int)
# 0.4839385624704414
# X_test.head(10)
final_predictions

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [119]:
output = pd.DataFrame({'PassengerId': testData.PassengerId,
                        'Survived': final_predictions})
output.to_csv('submission.csv', index=False)